# HASHING DE FEATURES
---
O conjunto de dados é estritamente categórico possuindo alta cardinalidade, estando sujeito também a adição e remoção de novos perfis e atribuições a qualquer momento.

Este trabalho objetiva construir um recomendador de perfis que:

1 - Não requeira qualquer tipo de ajuste manual dos dados processados quando ocorrerem modificações no conjunto de perfis existente e de suas atribuições.

2 - Consiga lidar com uma ordem de magnitude a mais dos perfis existentes (de milhares para milhões), pois isto possibilitaria realizar a recomendação de, além de perfis dos SAP, também de direitos controlados pelo IdentityIQ e grupos do Active Directory.

Tomado em conjunto os requisitos acima, constatou-se que a técnica conhecida como feature hashing associada ao uso de shingling atende ao cenário proposto.


In [1]:
import numpy as np
import os
import duckdb
import pandas as pd
import hashlib
import math
import scipy.sparse
from pprint import pprint

As variáveis de ambiente abaixo precisam ser configuradas antes da execução deste notebook. Vide o arquivo **setenv.ps1.example**

In [2]:
DATASET                     = os.environ['DATASET']
SHINGLER_N                  = int(os.environ['SHINGLER_N'])
HASHED_FEATURE_COUNT        = int(os.environ['HASHED_FEATURE_COUNT'])
HASHED_FEATURES             = os.environ['HASHED_FEATURES']
HASHED_FEATURES_IDX         = os.environ["HASHED_FEATURES_IDX"]

Leitura do conjunto de dados contendo as informações da base integrada de RH e as atribuições de perfis

In [3]:
dataset_df = pd.read_parquet(DATASET)
dataset_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86332 entries, 0 to 86331
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   chave_usuario            86300 non-null  object
 1   tipo_usuario             86332 non-null  object
 2   centro_custo             39577 non-null  object
 3   lotacao_topo             86332 non-null  object
 4   sigla_lotacao            86332 non-null  object
 5   nome_lotacao             86332 non-null  object
 6   cargo                    86332 non-null  object
 7   enfase                   86332 non-null  object
 8   funcao                   86332 non-null  object
 9   sindicato                39070 non-null  object
 10  area_rh                  85902 non-null  object
 11  imovel                   85902 non-null  object
 12  local_negocio            85900 non-null  object
 13  grupo_prestacao_servico  46192 non-null  object
 14  regime_trabalho          73068 non-nul

Abaixo, segue as configurações de que campos serão processados usando a técnica de *shingling*.

Constatamos que um mecanismo de pesos para os diferentes atributos pode eventualmente ser útil para realização do ajuste fino do classificador, mas dado a natureza exploratória desse trabalho, este mecanismo não foi utilizado.

In [4]:
ROLES_COLUMN                = "roles"
SHINGLED_FEATURES           = set([
    'sigla_lotacao',
    #'roles'  # CAUSOU VIZINHANÇAS SEM SENTIDO. NÃO É POSSÍVEL USAR SEM PESOS
])
EXCLUDED_FEATURES           = set([
    'chave_usuario',
])
DEFAULT_FEATURE_WEIGHT      = 1
FEATURE_WEIGHTS             = {'sigla_lotacao' : 1}

A função de *shingling* utilizada neste trabalho é a função definida abaixo. Ela basicamente "picota" uma string em múltiplas substrings de tamanho n deslocando-as sempre uma posição para a direita

In [5]:
def shingler(n, s, as_set=False):
    s = str(s)
    result = []
    l = len(s)
    acc = []
    
    if l == 1 or l <= n:
        return [s]
    
    for i in range(0, l-n+1):
        shingle = s[i:i+n]
        result.append(shingle)
            
    return set(result) if as_set else result

Segue abaixo um exemplo de saída para uma sigla de lotação resultando em 18 substrings

In [6]:
shingler(SHINGLER_N, "TIC/CORP/DSCESI/DS-FRJD")

['TIC/CO',
 'IC/COR',
 'C/CORP',
 '/CORP/',
 'CORP/D',
 'ORP/DS',
 'RP/DSC',
 'P/DSCE',
 '/DSCES',
 'DSCESI',
 'SCESI/',
 'CESI/D',
 'ESI/DS',
 'SI/DS-',
 'I/DS-F',
 '/DS-FR',
 'DS-FRJ',
 'S-FRJD']

Abaixo, o *shingler* foi executado para uma "lotação irmã" fictícia, resultando em 15 substrings

In [7]:
shingler(SHINGLER_N, "TIC/CORP/DSCESI/XXXX")

['TIC/CO',
 'IC/COR',
 'C/CORP',
 '/CORP/',
 'CORP/D',
 'ORP/DS',
 'RP/DSC',
 'P/DSCE',
 '/DSCES',
 'DSCESI',
 'SCESI/',
 'CESI/X',
 'ESI/XX',
 'SI/XXX',
 'I/XXXX']

Ambas as lotações possuem 11 substrings em comum. Se considerarmos cada substring como uma feature, é possível constatar que existe uma proximidade estrutural entre estas lotações, apesar de não serem iguais.

Esta funcionalidade é alcançada por meio do aumento da dimensionalidade do conjunto de dados. Este aumento, entretanto, é tratado pelo chamado *hashing trick* que será devidamente explicado na parte teórica deste trabalho

In [8]:
lotacao1 = shingler(SHINGLER_N, "TIC/CORP/DSCESI/DS-FRJD")
lotacao2 = shingler(SHINGLER_N, "TIC/CORP/DSCESI/XXXX")
len(set(lotacao1).intersection(set(lotacao2)))

11

A técnica de feature hashing necessita aplicar duas funções de hashing para um valor a ser convertido.

A primeira função de hashing é utilizada para determinar a posição em que um atributo categórico será armazenado.

A segunda função de hashing é utilizada para determinar o sinal do valor a ser armazenado na posição determinada pelo o primeiro hash calculado.

Por se tratar de um método probabílistico em que colisões podem ocorrer, o sinal introduzido pela segunda função ajuda na tolerância à colisões. Maiores detalhes na parte teórica do trabalho

In [9]:
def hash_string(s):
    buf        = str(s).encode()
    sha1       = hashlib.sha1(buf)
    md5        = hashlib.md5(buf)
    hash_value = int(sha1.hexdigest(), base=16)
    sign_value = int(md5.hexdigest(), base=16)
    sign       = -1 if abs(sign_value % 2) == 1 else 1 
    result     = sign * hash_value 
    return result

Abaixo, calculamos o hash de um feature categórico. O feature é codificado como **"&lt;nome do feature&gt;::&lt;valor do feature&gt;"**

In [10]:
n = 64
valor_original = "sigla_lotacao::TIC/CORP/DSCESI/DS-PDDS"
hash_lotacao = hash_string(valor_original)
hash_original_mod_n = abs(hash_lotacao) % n 
print(f"""
valor                  : {valor_original}
hash_lotacao           : {hash_lotacao}
abs(hash_lotacao) % {n} : {hash_original_mod_n}
""")


valor                  : sigla_lotacao::TIC/CORP/DSCESI/DS-PDDS
hash_lotacao           : 580521354414209253383885614425930233875981244812
abs(hash_lotacao) % 64 : 12



Abaixo, calculamos uma alteração em um feature que provoque uma colisão com o primeiro hash, mas não do segundo 

In [11]:
valor_inicial = "tipo_usuario::PRESTADOR DE SERVIÇO"
hash_colisao = None
chs = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!?.,"
for ch1 in chs:
    if hash_colisao is not None:
        break
    for ch2 in chs:
        if hash_colisao is not None:
            break
        for ch3 in chs:
            valor_final = valor_inicial + ch1 + ch2 + ch3
            hash = hash_string(valor_final)
            if hash < 0 and abs(hash) % n == hash_original_mod_n:
                hash_colisao = hash
                break
print(f"""
valor                  : {valor_final}
hash_colisao           : {hash_colisao}
abs(hash_colisao) % {n} : {abs(hash_colisao) % n}
""")


valor                  : tipo_usuario::PRESTADOR DE SERVIÇOafk
hash_colisao           : -152937679684757113643692380177006773294535406668
abs(hash_colisao) % 64 : 12



Neste cenário fictício, ambas as features seriam mapeadas para a posição 12 de um vetor de tamanho 16 (tamanho arbitrariamente reduzido para provocar colisão neste cenário artificial).

Neste caso, o efeito da colisão é mitigado pela diferença de sinais. Vide detalhes na parte teórica do trabalho.

Segue abaixo as funções utilizado para realizar o feature hashing com e sem *shingling*

In [12]:
def hash_feature(name, value):
    if value is None or value == "" or value == 0:
        return []
    feature = f"{name}::{value}"
    p = FEATURE_WEIGHTS.get(name, DEFAULT_FEATURE_WEIGHT)
    return feature, hash_string(feature), p

In [13]:
feature, hashes, p = hash_feature("sigla_lotacao", "TIC/CORP/DSCESI/DS-PDDS")
print(f"""
feature : {feature}
hashes  : {hashes}
p       : {p}
""")


feature : sigla_lotacao::TIC/CORP/DSCESI/DS-PDDS
hashes  : 580521354414209253383885614425930233875981244812
p       : 1



In [14]:
feature, hashes, p = hash_feature("tipo_usuario", "PRESTADOR DE SERVIÇOafk")
print(f"""
feature : {feature}
hashes  : {hashes}
p       : {p}
""")


feature : tipo_usuario::PRESTADOR DE SERVIÇOafk
hashes  : -152937679684757113643692380177006773294535406668
p       : 1



In [15]:
def hash_shingles(name, value, k):
    if value is None or value == "" or value == 0:
        return []
    
    hashes = []
    shingles = []
    for shingle in shingler(k, value):
        feature = f"{name}::{shingle}"
        h = hash_string(feature)
        shingles.append(feature)
        hashes.append(h)
    
    shingle_count = len(hashes)
    p = FEATURE_WEIGHTS.get(name, DEFAULT_FEATURE_WEIGHT)
    ps = [ p / shingle_count ] * shingle_count
    
    return list(zip(shingles, hashes, ps))

In [16]:
list(hash_shingles('sigla_lotacao', "TIC/CORP/DSCESI/DS-PDDS", SHINGLER_N))

[('sigla_lotacao::TIC/CO',
  -730274207718065717245553429308356210311740865032,
  0.05555555555555555),
 ('sigla_lotacao::IC/COR',
  -373633975827719757435700647505553931131855812956,
  0.05555555555555555),
 ('sigla_lotacao::C/CORP',
  1080979219743646486836742103071560908203812013451,
  0.05555555555555555),
 ('sigla_lotacao::/CORP/',
  -952594771626872110135713732559202401197871445227,
  0.05555555555555555),
 ('sigla_lotacao::CORP/D',
  -95097466763751454432359261244372763161453129758,
  0.05555555555555555),
 ('sigla_lotacao::ORP/DS',
  1239745038134983926731129470014433906122438776577,
  0.05555555555555555),
 ('sigla_lotacao::RP/DSC',
  239024900118180735701278668947112509290076429247,
  0.05555555555555555),
 ('sigla_lotacao::P/DSCE',
  1029606924163952446674767124603193492618110951838,
  0.05555555555555555),
 ('sigla_lotacao::/DSCES',
  -1110266309991503018367385692444359394226034410028,
  0.05555555555555555),
 ('sigla_lotacao::DSCESI',
  -24987178323783664671744920259053512

A função abaixo é a funçaõ que recebe um dicionário contendo os dados dos usuários assim como uma lista de roles (como um atributo multi-valorado) e retorna uma lista de tuplas no formato (nome da feature, hash da feature, peso da feature) utilizado para preenchimento da matriz numérica que será utilizada para cômputo posterior do KNN

In [17]:
def hash_row(row, k):
    result = []
    for key, val in row.items():
        if key in EXCLUDED_FEATURES:
            continue
        if key == ROLES_COLUMN:
            for role in val:
                if key in SHINGLED_FEATURES:
                    feature_hashes_p = hash_shingles("role", role, k)
                    result.extend(feature_hashes_p)
                feature_hash_p = hash_feature(role, 1)
                result.append(feature_hash_p)
        elif key in SHINGLED_FEATURES:
            feature_hashes_p = hash_shingles(key, val, k) 
            result.extend(feature_hashes_p)
        # roles are multivalued and always have the value 1
        else:        
            feature_hash_p = hash_feature(key, val)
            if feature_hash_p:
                result.append(feature_hash_p)
    return result

Segue abaixo a saída do *featurer hahser* & *shingler*. O número de features resultantes para este usuário é de 33.

Este número varia de usuário para usuário e é importante que seja utilizado uma representação esparsa da matriz de dados a serem processados.

In [18]:
row = dataset_df[dataset_df["chave_usuario"] == "U4UL"].to_dict(orient="records")[0]
#row = dataset_df[0:1].to_dict(orient="records")[0]
print("Chave do usuário processado: " + row["chave_usuario"])
hashes = hash_row(row, SHINGLER_N)
pprint(hashes, width=200)
len(hashes)

Chave do usuário processado: U4UL
[('tipo_usuario::EMPREGADO', -1143454999077322132947551267231288994892033570655, 1),
 ('centro_custo::ST75AT1B00', -1133144768198451777642148719479341277461087595357, 1),
 ('lotacao_topo::TIC', 883719330180455605187708388056895414972581655255, 1),
 ('sigla_lotacao::TIC/CO', -730274207718065717245553429308356210311740865032, 0.05555555555555555),
 ('sigla_lotacao::IC/COR', -373633975827719757435700647505553931131855812956, 0.05555555555555555),
 ('sigla_lotacao::C/CORP', 1080979219743646486836742103071560908203812013451, 0.05555555555555555),
 ('sigla_lotacao::/CORP/', -952594771626872110135713732559202401197871445227, 0.05555555555555555),
 ('sigla_lotacao::CORP/D', -95097466763751454432359261244372763161453129758, 0.05555555555555555),
 ('sigla_lotacao::ORP/DS', 1239745038134983926731129470014433906122438776577, 0.05555555555555555),
 ('sigla_lotacao::RP/DSC', 239024900118180735701278668947112509290076429247, 0.05555555555555555),
 ('sigla_lotacao::P/

33

A função abaixo recebe uma lista de hashes e os aplica em uma linha específica de uma matriz pré-alocada.

A lista de hashes é composta tuplas no formato: (nome da feature, hash computado, peso da feature)

Em caso de hash negativo, o sinal negativo é aplicado no peso da feature

In [19]:
def apply_hashes_into(hashes, row_idx, hashed_features, feature_count):
    for feature, hash, p in hashes:
        idx = abs(hash) % feature_count
        sign = -1 if hash < 0 else 1
        hashed_features[row_idx, idx] = sign * p

No exemplo abaixo, estamos alocando uma matriz com 1 linha e HASHED_FEATURE_COUNT colunas.

O tipo np.float16 (apesar de não possuir suporte para operações diretas em ponto flutuante na CPU, somente em GPU's) é usado para economizar memória

In [20]:
rows = 1
cols = HASHED_FEATURE_COUNT
shape = rows, cols
hashed_features = np.zeros(shape, dtype=np.float16)
apply_hashes_into(hashes, row_idx=0, hashed_features=hashed_features, feature_count=HASHED_FEATURE_COUNT)
np.count_nonzero(hashed_features)

33

Aqui alocamos uma matriz esparsa do tipo "dictionary of keys" para cada um dos 85.000+ funcionários da Petrobras e com HASHED_FEATURE_COUNT colunas.

Este tipo de matriz esparsa é utilizada pois é de rápida construção iterativa. Utilizamos o tipo **float16** para redução do consumo total de memória

In [21]:
rowcount = len(dataset_df)
colcount = HASHED_FEATURE_COUNT
shape = rowcount, colcount
hashed_features = scipy.sparse.dok_array(shape, np.float16) # dictionary of keys - efficient O(1) element access
hashed_features.shape

(86332, 500000)

O trecho de código abaixo aplica a função **apply_hashes_into** na matriz esparsa pré-alocada.

Ela utiliza uma série de **generator comprehensions** para processar os dados sem que os mesmos consumam memória como uma **list comprehension** consumiria. Como o mecanismo de iteração das **generator comprehensions** é escrito em C, frequentemente este tipo de processamento apresenta uma performance melhor do que o uso de laços *for* explicitamente criados.

In [22]:
column_names = dataset_df.columns
f = lambda row_idx, hashes: apply_hashes_into(hashes, row_idx, hashed_features, HASHED_FEATURE_COUNT)

# using generator compreehensions to save memory ... this is ugly, but uses less memory
records      = (  list(r)                     for r                   in dataset_df.itertuples(index=False) )
rows         = (  dict(zip(column_names, r))  for r                   in records                            )
hashes_rows  = (  hash_row(row, SHINGLER_N)   for row                 in rows                               )
apply_hashes = (  f(row_idx, hashes_row)      for row_idx, hashes_row in enumerate(hashes_rows)             )

for i, _ in enumerate(apply_hashes):
    # for loop only used for logging purposes
    if i % 5000 == 0:
        print(f"-> {i}/{rowcount}")
print(f"-> {rowcount}/{rowcount}")

-> 0/86332
-> 5000/86332
-> 10000/86332
-> 15000/86332
-> 20000/86332
-> 25000/86332
-> 30000/86332
-> 35000/86332
-> 40000/86332
-> 45000/86332
-> 50000/86332
-> 55000/86332
-> 60000/86332
-> 65000/86332
-> 70000/86332
-> 75000/86332
-> 80000/86332
-> 85000/86332
-> 86332/86332


A matriz preenchida é posteriormente salva no formato **compressed sparse row**. Este formato é utilizado pois possibilita o acesso a linhas individuais de maneira otimizada (necessário para o cômputo do KNN), assim como a possibilita a execução de operações aritméticas. Este é o formato que a biblioteca **PyNNDescent** utiliza para cômputo do KNN de maneira distribuída entre as múltiplas CPU's de um computador 

In [23]:
hashed_features = hashed_features.tocsr() # csr is fast for dot products and row slicing
scipy.sparse.save_npz(HASHED_FEATURES, hashed_features)

As informações contidas na matriz esparsa são apenas numéricas sendo necessário correlacionar uma posição da matriz com dados de um usuário. Estas informações são salvas abaixo em formato parquet para consumo posterior

In [24]:
hashed_features_df = pd.DataFrame({
    "index"          : dataset_df.index.tolist()
,   "chave_usuario"  : dataset_df["chave_usuario"]
,   "lotacao_topo"   : dataset_df["lotacao_topo"]
,   "sigla_lotacao"  : dataset_df["sigla_lotacao"]
,   "tipo_usuario"   : dataset_df["tipo_usuario"]
,   "cargo"          : dataset_df["cargo"]
,   "enfase"         : dataset_df["enfase"]
,   "funcao"         : dataset_df["funcao"]
})

hashed_features_df.to_parquet(HASHED_FEATURES_IDX)
hashed_features_df.head(100)

,index,chave_usuario,lotacao_topo,sigla_lotacao,tipo_usuario,cargo,enfase,funcao
0,0,A00D,LOEP,LOEP/LON/OPTER/TTOFF,PRESTADOR DE SERVIÇO,n/a,n/a,n/a
1,1,A00E,ISC,ISC/OSC/SCRIO/SCCRJ,PRESTADOR DE SERVIÇO,n/a,n/a,n/a
2,2,A00Q,SUB,SUB/SSUB/PGDO/PCIOP,PRESTADOR DE SERVIÇO,n/a,n/a,n/a
3,3,A00R,LOEP,LOEP/LOFF/EO,PRESTADOR DE SERVIÇO,n/a,n/a,n/a
4,4,A01B,UN-AM,UN-AM/ATP-U/DPCM,PRESTADOR DE SERVIÇO,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...
95,95,A0HO,UN-BUZ,UN-BUZ/SOP/GPS,PRESTADOR DE SERVIÇO,n/a,n/a,n/a
96,96,A0HW,LOEP,LOEP/LON/EO/RES,PRESTADOR DE SERVIÇO,n/a,n/a,n/a
97,97,A0I1,PGN,PGN/UN-AGN/APES/MI,EMPREGADO,PROF. PETROBRAS DE NIVEL TECNICO PLENO,PCR NT MANUTENCAO INSTRUMENTACAO,n/a
98,98,A0I2,PGN,PGN/UN-AGN/APES/MI,EMPREGADO,PROF. PETROBRAS DE NIVEL TECNICO PLENO,PCR NT MANUTENCAO INSTRUMENTACAO,n/a
